In [61]:
import pandas as pd

# 품목분류사례에 있는 HS 코드 100개 정답 데이터 만들기 + 4/6 자리 부모 코드 추가

### 품목분류사례 데이터에서 100개 코드 선정

In [62]:
crawling = pd.read_csv('/Users/zoohunn/Desktop/4-2/데사캡디/크롤링/품목분류사례_crawling_1005.csv')
crawling

,번호,시행일자,HSCode,이름,설명,사유
0,1,2025-08-28,8543.70-2090,ERGOBODY SHAPY EMS BELT; EB24-SB03;,"- 전극 패드가 내장된 직물 재질의 벨트와 컨트롤러, 충전 케이블이 지제 박스에 소...",- 관세율표 제8543호에는 ‘그 밖의 전기기기(이 류에 따로 분류되지 않은 것으로...
1,2,2025-08-28,3212.90-1000,"Colouring matter, put up in forms for retail s...","o Eosin-Y dye, Acetic acid, Isopropyl alcohol,...",o 관세율표 제3212호에는 “비수성(非水性) 매질(媒質)에 분산시킨 안료[금속 가...
2,3,2025-08-28,8543.70-2090,DEHUMIDIFICATION AND AROMATHERAPY MACHINE (Lav...,"- 향기 타블렛과 제습비즈를 본체에 삽입 후 팬을 구동시켜 방향․제습하고, UV L...","- 본 물품은 팬과 UV LED 램프가 결합된 본체와 향기타블렛, 제습비즈와 USB..."
3,4,2025-08-28,3212.90-1000,"Colouring matter, put up in forms for retail s...","o Eosin-Y dye, Phosphotungstic acid hydrate, L...",o 관세율표 제3212호에는 “비수성(非水性) 매질(媒質)에 분산시킨 안료[금속 가...
4,5,2025-08-28,7007.19-1000,SODALIME GLASS,"ㅇ 물품개요\n- 게임기 모니터의 전면부에 부착되는 Cover GLASS로, 이물질...",- 관세율표 제16부 총설에서 “다른 부에 보다 구체적으로 열거한 물품을 제외한 여...
...,...,...,...,...,...,...
29956,29957,2008-11-06,제2005.20-900,Potato powder preparation; Potato Pre Mix #3; ...,"감자 분말 86.2 %에 유장분말 8.1 %, 유당 4.2 %, 소금 1.5% 등을...","ㅇ 관세율표 제2005호에는“조제 또는 저장처리한 기타 채소”가 분류되며, 동호 해..."
29957,29958,2008-11-05,제0303.62-000,"Toothfish(Dissostichus spp.) neck meat, frozen...",- 파타고니아 이빨고기의 아가미와 몸체 사이의 목 부위 살을 절단하여 냉동한 것(육...,"ㅇ 관세율표 제0303호의 용어 ""냉동어류(제0304호의 어류의 피레트 및 기 타 ..."
29958,29959,2008-11-03,8538.90-9000,변경근거 관세청고시 2014-4호(2014.1.21)\n변경후 세번 8517.70-...,Touch panel VM283DZ1G,ㅇ 형태 및 구성\n- 상부전극(투명도전막(ITO)이 형성된 PET필름)과 하부전극...
29959,29960,2008-11-03,8538.90-9000,변경근거 관세청고시 2014-4호(2014.1.21)\n변경후 세번 8517.70-...,Window touch panel NT260YR1C,ㅇ 형태 및 구성\n- 상부전극(투명도전막(ITO)이 형성된 PET필름)과 하부전극...


In [63]:
df = pd.read_excel('/Users/zoohunn/Desktop/4-2/데사캡디/data/관세청_HS부호_20250101 (1).xlsx')
cols_to_keep = [
    'HS부호', 
    '적용시작일자', 
    '적용종료일자', 
    '한글품목명', 
    '영문품목명',  
    '성질통합분류코드명'
]

# df에서 한글품목명이 "기타"가 아닌 것만 필터링
df['HS부호'] = df['HS부호'].astype(str).str.zfill(10)
non_etc = df[df['한글품목명'] != '기타'].copy()

df_selected = non_etc[cols_to_keep].copy()
df_selected.head()

,HS부호,적용시작일자,적용종료일자,한글품목명,영문품목명,성질통합분류코드명
0,0101211000,20130101,20251231,농가 사육용,For farm breeding,(말)
2,0101291000,20120101,20251231,경주말,Horses for racing,(말)
4,0101300000,20220101,20251231,당나귀,Asses,(기타 산 동물)
6,0102211000,20170101,20251231,젖소,For milk,(소)
7,0102212000,20150101,20251231,육우(肉牛),For meat,(소)


In [64]:
# 10자리 HSCode 추출 (예: 8543.70-2090 형식)
crawling['HSCode_clean'] = crawling['HSCode'].astype(str).str.replace('[.-]', '', regex=True)
valid_codes = crawling[
    (crawling['HSCode_clean'].str.len() == 10) & 
    (crawling['HSCode_clean'].str.isdigit()) &
    (~crawling['HSCode'].astype(str).str.startswith('제'))
]['HSCode_clean'].drop_duplicates()

In [65]:
valid_codes

0        8543702090
1        3212901000
4        7007191000
6        8536504000
12       4202922000
            ...    
29879    0303420000
29896    9030909010
29903    8486209390
29922    8542333000
29929    8529909640
Name: HSCode_clean, Length: 3435, dtype: object

In [66]:
# 10자리로 정규화
valid_codes_clean = valid_codes.str.zfill(10)

In [67]:
# 크롤링에 있으면서 "기타"가 아닌 코드
available_codes = non_etc[non_etc['HS부호'].isin(valid_codes_clean)]['HS부호'].unique()

In [68]:
# 랜덤 100개 샘플링
sampled_codes = pd.Series(available_codes).sample(n=min(100, len(available_codes)), random_state=42)
sampled_codes

1604    8515211010
1906    9503003700
70      0813500000
1894    9503001900
1731    8609003000
           ...    
297     2303100000
862     5603930000
99      1103192000
1618    8518101000
807     5305001000
Length: 100, dtype: object

In [69]:
# 최종 결과
result = df_selected[df_selected['HS부호'].isin(sampled_codes)]
result

,HS부호,적용시작일자,적용종료일자,한글품목명,영문품목명,성질통합분류코드명
763,0409000000,20020101,20251231,천연꿀,Natural honey.,(기타 축산물)
1047,0714102020,20130101,20251231,펠릿(pellet),Pellets,(기타 채소류)
1101,0804502000,20130101,20251231,망고(mango),Mangoes,(망고)
1153,0813402000,20020101,20251231,대추,Jujubes,(대추)
1155,0813500000,20130101,20251231,이 류의 견과류나 건조한 과실의 혼합물,Mixtures of nuts or dried fruits of this Chapter,(기타 과실류)
...,...,...,...,...,...,...
11916,9032102000,20130101,20251231,고정식,Of fixed type,(기타 정밀기계)
11936,9032899010,20130101,20251231,항공기용,For aircraft,(기타 항공기 부품)
12151,9401200000,20130101,20251231,차량용 의자,Seats of a kind used for motor vehicles,(자동차 부품)
12241,9503001900,20070101,20251231,부분품과 부속품,Parts and accessories,(완구류)


In [70]:
# 모두 10자리인지 확인
print(result['HS부호'].astype(str).str.len().eq(10).all())

True


### 해당 100개코드에 대한 품목분류사례 변수 추가

In [71]:
# 먼저 크롤링 데이터의 컬럼 확인
print("크롤링 데이터 컬럼:", crawling.columns.tolist())

크롤링 데이터 컬럼: ['번호', '시행일자', 'HSCode', '이름', '설명', '사유', 'HSCode_clean']


In [73]:
crawling.head()

,번호,시행일자,HSCode,이름,설명,사유,HSCode_clean
0,1,2025-08-28,8543.70-2090,ERGOBODY SHAPY EMS BELT; EB24-SB03;,"- 전극 패드가 내장된 직물 재질의 벨트와 컨트롤러, 충전 케이블이 지제 박스에 소...",- 관세율표 제8543호에는 ‘그 밖의 전기기기(이 류에 따로 분류되지 않은 것으로...,8543702090
1,2,2025-08-28,3212.90-1000,"Colouring matter, put up in forms for retail s...","o Eosin-Y dye, Acetic acid, Isopropyl alcohol,...",o 관세율표 제3212호에는 “비수성(非水性) 매질(媒質)에 분산시킨 안료[금속 가...,3212901000
2,3,2025-08-28,8543.70-2090,DEHUMIDIFICATION AND AROMATHERAPY MACHINE (Lav...,"- 향기 타블렛과 제습비즈를 본체에 삽입 후 팬을 구동시켜 방향․제습하고, UV L...","- 본 물품은 팬과 UV LED 램프가 결합된 본체와 향기타블렛, 제습비즈와 USB...",8543702090
3,4,2025-08-28,3212.90-1000,"Colouring matter, put up in forms for retail s...","o Eosin-Y dye, Phosphotungstic acid hydrate, L...",o 관세율표 제3212호에는 “비수성(非水性) 매질(媒質)에 분산시킨 안료[금속 가...,3212901000
4,5,2025-08-28,7007.19-1000,SODALIME GLASS,"ㅇ 물품개요\n- 게임기 모니터의 전면부에 부착되는 Cover GLASS로, 이물질...",- 관세율표 제16부 총설에서 “다른 부에 보다 구체적으로 열거한 물품을 제외한 여...,7007191000


In [74]:
crawling_subset = crawling[['HSCode_clean','이름', '설명', '사유']].drop_duplicates(subset='HSCode_clean')

# result와 merge
result = result.merge(
    crawling_subset,
    left_on='HS부호',
    right_on='HSCode_clean',
    how='left'
).drop('HSCode_clean', axis=1)

# 결과 확인
print(f"\n추가된 컬럼: {result.columns.tolist()}")


추가된 컬럼: ['HS부호', '적용시작일자', '적용종료일자', '한글품목명', '영문품목명', '성질통합분류코드명', '이름', '설명', '사유']


In [75]:
result.head()

,HS부호,적용시작일자,적용종료일자,한글품목명,영문품목명,성질통합분류코드명,이름,설명,사유
0,0409000000,20020101,20251231,천연꿀,Natural honey.,(기타 축산물),Braided twine of polyethylene; LEGERO HS PE X8...,- 폴리에틸렌 멀티필라멘트사 8가닥을 조밀하게 엮어 만든 끈을 스 풀에 감은 것(횡...,○ 관세율표 제5607호에 “끈·배의 밧줄(cordage)·케이블[엮거나 짠 것인지...
1,0714102020,20130101,20251231,펠릿(pellet),Pellets,(기타 채소류),Manioc(cassava) pellet; CASSAVA RESIDUE PELLET,매니옥(카사바)에서 전분을 추출하고 남은 부산물을 펠릿 모양으로 성형한 것 (건조물...,ㅇ 관세율표 제0714호에는 “매니옥(manioc)ㆍ칡뿌리ㆍ살렙(salep)ㆍ돼지감...
2,0804502000,20130101,20251231,망고(mango),Mangoes,(망고),"Mangoes, dried; golden mango",ㅇ 껍질과 씨를 제거한 망고를 절단하여 동결건조한 황색계 불규칙한 조각을 플라스틱 ...,ㅇ 관세율표 제0804호에는 “대추야자ㆍ무화과ㆍ파인애플ㆍ아보카도(avocado) ㆍ...
3,0813402000,20020101,20251231,대추,Jujubes,(대추),"Jujube, dried; Red Dates Slices",ㅇ 씨를 제거하고 폭 방향으로 슬라이스하여 건조한 대추를 수지제 봉지에 소매포장한 ...,ㅇ 관세율표 제0813호에는 “건조한 과실(제0801호부터 제0806호까지에 해당하...
4,0813500000,20130101,20251231,이 류의 견과류나 건조한 과실의 혼합물,Mixtures of nuts or dried fruits of this Chapter,(기타 과실류),Mixtures of nuts or dried fruits of Chapter 8;...,ㅇ 건조한 대추의 한 면을 세로로 절단하여 씨를 제거하고 호두 조각을 넣어 낱개 포...,ㅇ 관세율표 제0813호에는 “건조한 과실(제0801호부터 제0806호까지에 해당하...


### all_hscode에서 부모 코드 가져오기

In [77]:
# all_hscode 로드
all_hscode = pd.read_csv('/Users/zoohunn/Desktop/4-2/데사캡디/data/all_hscode (1).csv')

# 코드 길이별로 정규화 (홀수면 앞에 0 추가)
all_hscode['code'] = all_hscode['code'].astype(str)

In [78]:
# 4자리 코드 데이터 (길이 3 또는 4)
hscode_4 = all_hscode[all_hscode['code'].str.len().isin([3, 4])].copy()
hscode_4['code_4'] = hscode_4['code'].str.zfill(4)
hscode_4_subset = hscode_4[['code_4', 'description', 'category']].drop_duplicates(subset='code_4')
hscode_4_subset = hscode_4_subset.rename(columns={
    'description': 'description_4자리', 
    'category': 'category_4자리'
})

# 6자리 코드 데이터 (길이 5 또는 6)
hscode_6 = all_hscode[all_hscode['code'].str.len().isin([5, 6])].copy()
hscode_6['code_6'] = hscode_6['code'].str.zfill(6)
hscode_6_subset = hscode_6[['code_6', 'description', 'category']].drop_duplicates(subset='code_6')
hscode_6_subset = hscode_6_subset.rename(columns={
    'description': 'description_6자리', 
    'category': 'category_6자리'
})

In [79]:
# result에서 4자리, 6자리 추출
result['HS_4자리'] = result['HS부호'].str[:4]
result['HS_6자리'] = result['HS부호'].str[:6]

In [80]:
# merge
result = result.merge(hscode_4_subset, left_on='HS_4자리', right_on='code_4', how='left')
result = result.merge(hscode_6_subset, left_on='HS_6자리', right_on='code_6', how='left')

In [81]:
# 임시 컬럼 제거
result = result.drop(['code_4', 'code_6'], axis=1)


In [82]:
result.head()

,HS부호,적용시작일자,적용종료일자,한글품목명,영문품목명,성질통합분류코드명,이름,설명,사유,HS_4자리,HS_6자리,description_4자리,category_4자리,description_6자리,category_6자리
0,0409000000,20020101,20251231,천연꿀,Natural honey.,(기타 축산물),Braided twine of polyethylene; LEGERO HS PE X8...,- 폴리에틸렌 멀티필라멘트사 8가닥을 조밀하게 엮어 만든 끈을 스 풀에 감은 것(횡...,○ 관세율표 제5607호에 “끈·배의 밧줄(cordage)·케이블[엮거나 짠 것인지...,0409,040900,Honey; natural,honey,Honey; natural,honey
1,0714102020,20130101,20251231,펠릿(pellet),Pellets,(기타 채소류),Manioc(cassava) pellet; CASSAVA RESIDUE PELLET,매니옥(카사바)에서 전분을 추출하고 남은 부산물을 펠릿 모양으로 성형한 것 (건조물...,ㅇ 관세율표 제0714호에는 “매니옥(manioc)ㆍ칡뿌리ㆍ살렙(salep)ㆍ돼지감...,0714,071410,"Manioc, arrowroot, salep, Jerusalem artichokes...","content, roots, similar, arrowroot, starch, je...","Vegetable roots and tubers; manioc (cassava), ...","tubers, vegetable, roots"
2,0804502000,20130101,20251231,망고(mango),Mangoes,(망고),"Mangoes, dried; golden mango",ㅇ 껍질과 씨를 제거한 망고를 절단하여 동결건조한 황색계 불규칙한 조각을 플라스틱 ...,ㅇ 관세율표 제0804호에는 “대추야자ㆍ무화과ㆍ파인애플ㆍ아보카도(avocado) ㆍ...,0804,080450,"Dates, figs, pineapples, avocados, guavas, man...","mangosteens, pineapples, mangoes, guavas, dates","Fruit, edible; guavas, mangoes and mangosteens...",fruit
3,0813402000,20020101,20251231,대추,Jujubes,(대추),"Jujube, dried; Red Dates Slices",ㅇ 씨를 제거하고 폭 방향으로 슬라이스하여 건조한 대추를 수지제 봉지에 소매포장한 ...,ㅇ 관세율표 제0813호에는 “건조한 과실(제0801호부터 제0806호까지에 해당하...,0813,081340,"Fruit, dried, other than that of heading no. 0...",fruit,"Fruit, edible; fruit n.e.c. in heading no. 081...",fruit
4,0813500000,20130101,20251231,이 류의 견과류나 건조한 과실의 혼합물,Mixtures of nuts or dried fruits of this Chapter,(기타 과실류),Mixtures of nuts or dried fruits of Chapter 8;...,ㅇ 건조한 대추의 한 면을 세로로 절단하여 씨를 제거하고 호두 조각을 넣어 낱개 포...,ㅇ 관세율표 제0813호에는 “건조한 과실(제0801호부터 제0806호까지에 해당하...,0813,081350,"Fruit, dried, other than that of heading no. 0...",fruit,"Nuts, edible; mixtures of nuts or dried fruits...",nuts


In [83]:
result.to_csv('/Users/zoohunn/Desktop/4-2/데사캡디/data/품목분류_100sample_원본.csv', index=False)

In [86]:
def combine_text(*args):
    """빈 값 제외하고 텍스트 합치기"""
    return ' '.join([str(x) for x in args if pd.notna(x) and str(x).strip()])

# 사용자_상품명
result['사용자_상품명'] = result.apply(
    lambda row: combine_text(row['한글품목명'], row['영문품목명'], row['이름']), 
    axis=1
)

# 사용자_상품설명
def make_description(row):
    parts = []
    
    # 대분류
    if pd.notna(row['성질통합분류코드명']):
        parts.append(f"[대분류]\n성질통합분류: {row['성질통합분류코드명']}")
    if pd.notna(row['description_4자리']):
        parts.append(f"4자리 설명: {row['description_4자리']}")
    if pd.notna(row['category_4자리']):
        parts.append(f"4자리 카테고리: {row['category_4자리']}")
    
    # 중분류
    if pd.notna(row['description_6자리']):
        parts.append(f"\n[중분류]\n6자리 설명: {row['description_6자리']}")
    if pd.notna(row['category_6자리']):
        parts.append(f"6자리 카테고리: {row['category_6자리']}")
    
    # 소분류
    if pd.notna(row['설명']):
        parts.append(f"\n[소분류]\n상세 설명: {row['설명']}")
    if pd.notna(row['사유']):
        parts.append(f"분류 사유: {row['사유']}")
    
    return '\n'.join(parts)

result['사용자_상품설명'] = result.apply(make_description, axis=1)


In [88]:
result.to_csv('/Users/zoohunn/Desktop/4-2/데사캡디/data/품목분류_100sample_사용자입력.csv', index=False)